# 🎓 The Virtual NMR Spectrometer

Welcome to the **Interactive Relaxation Lab**! 

This notebook simulates a **Nuclear Magnetic Resonance (NMR)** experiment on a virtual protein.
NMR relaxation rates ($R_1$, $R_2$, and Heteronuclear NOE) are the primary tools biophysicists use to study protein movement at the atomic level.

### 🎯 Goal
Understand how **molecular size** (tumbling rate) and **magnetic field strength** affect the signals we measure. This simulation uses the standard *Lipari-Szabo Model-Free* formalism, which is the gold standard for analyzing protein dynamics.

In [ ]:
# ⚙️ Install Dependencies
# We install openmm via pip for compatibility.
# We force reinstall synth-pdb from GitHub to avoid caching old versions.
try:
    import google.colab
    print("Installing OpenMM and synth-pdb...")
    !pip install -q openmm matplotlib ipywidgets py3Dmol
    !pip install -q --force-reinstall --no-cache-dir git+https://github.com/elkins/synth-pdb.git@master
except ImportError:
    print("Assuming local environment.")
    pass


In [ ]:
# ⚙️ HOTFIX: Patch synth_pdb for Colab compatibility
# This cell ensures the code handles missing OpenMM gracefully.
import os
import synth_pdb
import base64

package_dir = os.path.dirname(synth_pdb.__file__)
physics_file = os.path.join(package_dir, "physics.py")

print(f"🚨 Applying robust hotfix to: {physics_file}")

encoded_patch = "aW1wb3J0IGxvZ2dpbmcKdHJ5OgogICAgaW1wb3J0IG9wZW5tbS5hcHAgYXMgYXBwCiAgICBpbXBvcnQgb3Blbm1tIGFzIG1tCiAgICBmcm9tIG9wZW5tbSBpbXBvcnQgdW5pdAogICAgSEFTX09QRU5NTSA9IFRydWUKZXhjZXB0IEltcG9ydEVycm9yOgogICAgSEFTX09QRU5NTSA9IEZhbHNlCiAgICBhcHAgPSBOb25lCiAgICBtbSA9IE5vbmUKICAgIHVuaXQgPSBOb25lCmltcG9ydCBzeXMKaW1wb3J0IG9zCgpsb2dnZXIgPSBsb2dnaW5nLmdldExvZ2dlcihfX25hbWVfXykKCmNsYXNzIEVuZXJneU1pbmltaXplcjoKICAgICIiIgogICAgUGVyZm9ybXMgZW5lcmd5IG1pbmltaXphdGlvbiBvbiBtb2xlY3VsYXIgc3RydWN0dXJlcyB1c2luZyBPcGVuTU0uCiAgICAKICAgICMjIyBFZHVjYXRpb25hbCBOb3RlOiBXaGF0IGlzIEVuZXJneSBNaW5pbWl6YXRpb24/CiAgICAtLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLQogICAgUHJvdGVpbnMgZm9sZCBpbnRvIHNwZWNpZmljIDNEIHNoYXBlcyB0byBtaW5pbWl6ZSB0aGVpciAiR2liYnMgRnJlZSBFbmVyZ3kiLgogICAgQSBnZW5lcmF0ZWQgc3RydWN0dXJlIChsaWtlIG9uZSBidWlsdCBmcm9tIHNpbXBsZSBnZW9tZXRyeSkgb2Z0ZW4gaGFzICJjbGFzaGVzIgogICAgd2hlcmUgYXRvbXMgYXJlIHRvbyBjbG9zZSAoaGlnaCBWYW4gZGVyIFdhYWxzIHJlcHVsc2lvbikgb3IgYm9uZCBhbmdsZXMgYXJlIHN0cmFpbmVkLgogICAgCiAgICBFbmVyZ3kgTWluaW1pemF0aW9uIGlzIGxpa2Ugcm9sbGluZyBhIGJhbGwgZG93biBhIGhpbGwuIFRoZSAiRW5lcmd5IExhbmRzY2FwZSIKICAgIHJlcHJlc2VudHMgdGhlIHBvdGVudGlhbCBlbmVyZ3kgb2YgdGhlIHByb3RlaW4gYXMgYSBmdW5jdGlvbiBvZiBhbGwgaXRzIGF0b20gY29vcmRpbmF0ZXMuCiAgICBUaGUgYWxnb3JpdGhtIG1vdmVzIGF0b21zIHNsaWdodGx5IHRvIHJlZHVjZSB0aGlzIGVuZXJneSwgZmluZGluZyBhIGxvY2FsIG1pbmltdW0KICAgIHdoZXJlIHRoZSBzdHJ1Y3R1cmUgaXMgcGh5c2ljYWxseSByZWxheGVkLgoKICAgICMjIyBOTVIgUGVyc3BlY3RpdmU6CiAgICBJbiBOTVIgc3RydWN0dXJlIGNhbGN1bGF0aW9uIChlLmcuLCBDWUFOQSwgWFBMT1ItTklIKSwgbWluaW1pemF0aW9uIGlzIG9mdGVuIHBhcnQgb2YKICAgICJTaW11bGF0ZWQgQW5uZWFsaW5nIi4gU3RydWN0dXJlcyBhcmUgY2FsY3VsYXRlZCB0byBzYXRpc2Z5IGV4cGVyaW1lbnRhbCByZXN0cmFpbnRzCiAgICAoTk9FcywgSi1jb3VwbGluZ3MpIGFuZCB0aGVuIGVuZXJneS1taW5pbWl6ZWQgdG8gZW5zdXJlIGdvb2QgZ2VvbWV0cnkuCiAgICBUaGlzIG1vZHVsZSBwZXJmb3JtcyB0aGF0IGZpbmFsICJnZW9tZXRyeSByZWd1bGFyaXphdGlvbiIgc3RlcC4KICAgICIiIgogICAgCiAgICBkZWYgX19pbml0X18oc2VsZiwgZm9yY2VmaWVsZF9uYW1lPSdhbWJlcjE0LWFsbC54bWwnLCBzb2x2ZW50X21vZGVsPU5vbmUpOgogICAgICAgICIiIgogICAgICAgIEluaXRpYWxpemUgdGhlIE1pbmltaXplciB3aXRoIGEgRm9yY2VmaWVsZCBhbmQgU29sdmVudCBNb2RlbC4KICAgICAgICAKICAgICAgICBBcmdzOgogICAgICAgICAgICBmb3JjZWZpZWxkX25hbWU6IFRoZSAicnVsZWJvb2siIGZvciBob3cgYXRvbXMgaW50ZXJhY3QuCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgJ2FtYmVyMTQtYWxsLnhtbCcgZGVzY3JpYmVzIHByb3RlaW4gYXRvbXMgKHBhcmFtZXRlcnMgZm9yIGJvbmQgbGVuZ3RocywKICAgICAgICAgICAgICAgICAgICAgICAgICAgICBhbmdsZXMsIGNoYXJnZXMsIGFuZCBWZFcgcmFkaWkpLgogICAgICAgICAgICBzb2x2ZW50X21vZGVsOiAgIEhvdyB3YXRlciBpcyBzaW11bGF0ZWQuIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICdhcHAuT0JDMicgaXMgYW4gIkltcGxpY2l0IFNvbHZlbnQiIG1vZGVsLiBJbnN0ZWFkIG9mIHNpbXVsYXRpbmcKICAgICAgICAgICAgICAgICAgICAgICAgICAgICB0aG91c2FuZHMgb2YgaW5kaXZpZHVhbCB3YXRlciBtb2xlY3VsZXMgKEV4cGxpY2l0IFNvbHZlbnQpLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgIGl0IHVzZXMgYSBtYXRoZW1hdGljYWwgY29udGludXVtIHRvIGFwcHJveGltYXRlIHdhdGVyJ3MgZGllbGVjdHJpYyAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICBzaGllbGRpbmcgYW5kIGh5ZHJvcGhvYmljIGVmZmVjdHMuIFRoaXMgaXMgbXVjaCBmYXN0ZXIuCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIyMjIE5NUiBOb3RlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgIFNpbmNlIE5NUiBpcyBwZXJmb3JtZWQgaW4gc29sdXRpb24gKG5vdCBjcnlzdGFscyksIGltcGxpY2l0IHNvbHZlbnQgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgYWltcyB0byBhcHByb3hpbWF0ZSB0aGF0IHNvbHV0aW9uIGVudmlyb25tZW50LCBkaXN0aW5jdCBmcm9tIHRoZQogICAgICAgICAgICAgICAgICAgICAgICAgICAgIHZhY3V1bSBvciBjcnlzdGFsIGxhdHRpY2UgYXNzdW1wdGlvbnMgb2Ygb3RoZXIgbWV0aG9kcy4KICAgICAgICAiIiIKICAgICAgICBpZiBub3QgSEFTX09QRU5NTToKICAgICAgICAgICAgICMgV2UgYWxsb3cgaW5pdGlhbGl6YXRpb24gdG8gcGFzcyBzbyB0aGUgbW9kdWxlIGNhbiBiZSBpbXBvcnRlZAogICAgICAgICAgICAgIyBidXQgbWV0aG9kcyB3aWxsIGNoZWNrIEhBU19PUEVOTU0KICAgICAgICAgICAgIHBhc3MKCiAgICAgICAgIyBTZXQgZGVmYXVsdCBzb2x2ZW50IG1vZGVsCiAgICAgICAgaWYgc29sdmVudF9tb2RlbCBpcyBOb25lIGFuZCBIQVNfT1BFTk1NOgogICAgICAgICAgICBzb2x2ZW50X21vZGVsID0gYXBwLk9CQzIKCiAgICAgICAgc2VsZi5mb3JjZWZpZWxkX25hbWUgPSBmb3JjZWZpZWxkX25hbWUKICAgICAgICBzZWxmLndhdGVyX21vZGVsID0gJ2FtYmVyMTQvdGlwM3BmYi54bWwnIAogICAgICAgIHNlbGYuc29sdmVudF9tb2RlbCA9IHNvbHZlbnRfbW9kZWwKICAgICAgICAKICAgICAgICBpZiBub3QgSEFTX09QRU5NTToKICAgICAgICAgICAgcmV0dXJuCgogICAgICAgICMgTWFwIHNvbHZlbnQgbW9kZWxzIHRvIHRoZWlyIHBhcmFtZXRlciBmaWxlcyBpbiBPcGVuTU0KICAgICAgICAjIFRoZXNlIG5lZWQgdG8gYmUgbG9hZGVkIGFsb25nc2lkZSB0aGUgbWFpbiBmb3JjZWZpZWxkCiAgICAgICAgIyBTdGFuZGFyZCBPcGVuTU0gcGF0aHMgb2Z0ZW4gaGF2ZSAnaW1wbGljaXQvJyBhdCB0aGUgcm9vdAogICAgICAgIHNvbHZlbnRfeG1sX21hcCA9IHsKICAgICAgICAgICAgYXBwLk9CQzI6ICdpbXBsaWNpdC9vYmMyLnhtbCcsCiAgICAgICAgICAgIGFwcC5PQkMxOiAnaW1wbGljaXQvb2JjMS54bWwnLAogICAgICAgICAgICBhcHAuR0JuOiAgJ2ltcGxpY2l0L2dibi54bWwnLAogICAgICAgICAgICBhcHAuR0JuMjogJ2ltcGxpY2l0L2dibjIueG1sJywKICAgICAgICAgICAgYXBwLkhDVDogICdpbXBsaWNpdC9oY3QueG1sJywKICAgICAgICB9CgogICAgICAgICMgQnVpbGQgbGlzdCBvZiBYTUwgZmlsZXMgdG8gbG9hZAogICAgICAgIGZmX2ZpbGVzID0gW3NlbGYuZm9yY2VmaWVsZF9uYW1lLCBzZWxmLndhdGVyX21vZGVsXQogICAgICAgIAogICAgICAgIGlmIHNlbGYuc29sdmVudF9tb2RlbCBpbiBzb2x2ZW50X3htbF9tYXA6CiAgICAgICAgICAgIGZmX2ZpbGVzLmFwcGVuZChzb2x2ZW50X3htbF9tYXBbc2VsZi5zb2x2ZW50X21vZGVsXSkKICAgICAgICAKICAgICAgICB0cnk6CiAgICAgICAgICAgICMgVGhlIEZvcmNlRmllbGQgb2JqZWN0IGxvYWRzIHRoZSBkZWZpbml0aW9ucyBvZiBhdG9tIHR5cGVzIGFuZCBwYXJhbWV0ZXJzLgogICAgICAgICAgICBzZWxmLmZvcmNlZmllbGQgPSBhcHAuRm9yY2VGaWVsZCgqZmZfZmlsZXMpCiAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICBsb2dnZXIuZXJyb3IoZiJGYWlsZWQgdG8gbG9hZCBmb3JjZWZpZWxkIHtmZl9maWxlc306IHtlfSIpCiAgICAgICAgICAgIHJhaXNlCgogICAgZGVmIG1pbmltaXplKHNlbGYsIHBkYl9maWxlX3BhdGg6IHN0ciwgb3V0cHV0X3BhdGg6IHN0ciwgbWF4X2l0ZXJhdGlvbnM6IGludCA9IDAsIHRvbGVyYW5jZTogZmxvYXQgPSAxMC4wKSAtPiBib29sOgogICAgICAgICIiIgogICAgICAgIE1pbmltaXplcyB0aGUgZW5lcmd5IG9mIGEgc3RydWN0dXJlIGFscmVhZHkgY29udGFpbmluZyBjb3JyZWN0IGF0b21zIChpbmNsdWRpbmcgSHlkcm9nZW5zKS4KICAgICAgICAKICAgICAgICBBcmdzOgogICAgICAgICAgICBwZGJfZmlsZV9wYXRoOiBJbnB1dCBQREIgcGF0aC4KICAgICAgICAgICAgb3V0cHV0X3BhdGg6IE91dHB1dCBQREIgcGF0aC4KICAgICAgICAgICAgbWF4X2l0ZXJhdGlvbnM6IExpbWl0IHN0ZXBzICgwID0gdW50aWwgY29udmVyZ2VuY2UpLgogICAgICAgICAgICB0b2xlcmFuY2U6IFRhcmdldCBlbmVyZ3kgY29udmVyZ2VuY2UgdGhyZXNob2xkIChrSi9tb2wpLgogICAgICAgICIiIgogICAgICAgIGlmIG5vdCBIQVNfT1BFTk1NOgogICAgICAgICAgICAgbG9nZ2VyLmVycm9yKCJDYW5ub3QgbWluaW1pemU6IE9wZW5NTSBub3QgZm91bmQuIikKICAgICAgICAgICAgIHJldHVybiBGYWxzZQoKICAgICAgICAjIFRoaXMgbWV0aG9kIGFzc3VtZXMgdGhlIGlucHV0IFBEQiBpcyBwZXJmZWN0IChoYXMgSHlkcm9nZW5zLCBjb3JyZWN0IG5hbWVzKS4KICAgICAgICAjIFNlZSAnYWRkX2h5ZHJvZ2Vuc19hbmRfbWluaW1pemUnIGZvciB0aGUgcm9idXN0IHZlcnNpb24gdXNlZCBieSBzeW50aC1wZGIuCiAgICAgICAgcmV0dXJuIHNlbGYuX3J1bl9zaW11bGF0aW9uKHBkYl9maWxlX3BhdGgsIG91dHB1dF9wYXRoLCBtYXhfaXRlcmF0aW9ucywgdG9sZXJhbmNlLCBhZGRfaHlkcm9nZW5zPUZhbHNlKQoKICAgIGRlZiBhZGRfaHlkcm9nZW5zX2FuZF9taW5pbWl6ZShzZWxmLCBwZGJfZmlsZV9wYXRoOiBzdHIsIG91dHB1dF9wYXRoOiBzdHIpIC0+IGJvb2w6CiAgICAgICAgIiIiCiAgICAgICAgUm9idXN0IG1pbmltaXphdGlvbiBwaXBlbGluZTogQWRkcyBIeWRyb2dlbnMgLT4gQ3JlYXRlcy9NaW5pbWl6ZXMgU3lzdGVtIC0+IFNhdmVzIFJlc3VsdC4KICAgICAgICAKICAgICAgICAjIyMgV2h5IEFkZCBIeWRyb2dlbnM/CiAgICAgICAgWC1yYXkgY3J5c3RhbGxvZ3JhcGh5IG9mdGVuIGRvZXNuJ3QgcmVzb2x2ZSBoeWRyb2dlbiBhdG9tcyBiZWNhdXNlIHRoZXkgaGF2ZSB2ZXJ5IGZldyBlbGVjdHJvbnMuCiAgICAgICAgSG93ZXZlciwgTW9sZWN1bGFyIER5bmFtaWNzIGZvcmNlZmllbGRzIChsaWtlIEFtYmVyKSBhcmUgZXhwbGljaXRseSAiQWxsLUF0b20iLiBUaGV5IFJFUVVJUkUKICAgICAgICBoeWRyb2dlbnMgdG8gY2FsY3VsYXRlIGJvbmQgYW5nbGVzIGFuZCBlbGVjdHJvc3RhdGljcyAoaC1ib25kcykgY29ycmVjdGx5LgogICAgICAgIAogICAgICAgICMjIyBOTVIgUGVyc3BlY3RpdmU6CiAgICAgICAgVW5saWtlIFgtcmF5LCBOTVIgcmVsaWVzIGVudGlyZWx5IG9uIHRoZSBtYWduZXRpYyBzcGluIG9mIHByb3RvbnMgKEgxKS4gSHlkcm9nZW5zIGFyZQogICAgICAgIHRoZSAiZXllcyIgb2YgTk1SLiBDb3JyZWN0bHkgcGxhY2luZyB0aGVtIGlzIGNyaXRpY2FsIG5vdCBqdXN0IGZvciBwaHlzaWNzIGJ1dCBmb3IKICAgICAgICBwcmVkaWN0aW5nIE5PRXMgKE51Y2xlYXIgT3ZlcmhhdXNlciBFZmZlY3RzKSB3aGljaCBkZXBlbmQgb24gSC1IIGRpc3RhbmNlcy4KICAgICAgICBXZSB1c2UgYGFwcC5Nb2RlbGxlcmAgdG8gImd1ZXNzIiB0aGUgc3RhbmRhcmQgcG9zaXRpb25zIG9mIGh5ZHJvZ2VucyBhdCBzcGVjaWZpYyBwSCAoNy4wKS4KICAgICAgICAiIiIKICAgICAgICBpZiBub3QgSEFTX09QRU5NTToKICAgICAgICAgICAgIGxvZ2dlci5lcnJvcigiQ2Fubm90IGFkZCBoeWRyb2dlbnM6IE9wZW5NTSBub3QgZm91bmQuIikKICAgICAgICAgICAgIHJldHVybiBGYWxzZQogICAgICAgICAgICAgCiAgICAgICAgcmV0dXJuIHNlbGYuX3J1bl9zaW11bGF0aW9uKHBkYl9maWxlX3BhdGgsIG91dHB1dF9wYXRoLCBhZGRfaHlkcm9nZW5zPVRydWUpCgogICAgZGVmIF9ydW5fc2ltdWxhdGlvbihzZWxmLCBpbnB1dF9wYXRoLCBvdXRwdXRfcGF0aCwgbWF4X2l0ZXJhdGlvbnM9MCwgdG9sZXJhbmNlPTEwLjAsIGFkZF9oeWRyb2dlbnM9VHJ1ZSk6CiAgICAgICAgIiIiSW50ZXJuYWwgd29ya2VyIGZvciBzZXR0aW5nIHVwIGFuZCBydW5uaW5nIHRoZSBPcGVuTU0gU2ltdWxhdGlvbi4iIiIKICAgICAgICBsb2dnZXIuaW5mbyhmIlByb2Nlc3NpbmcgcGh5c2ljcyBmb3Ige2lucHV0X3BhdGh9Li4uIikKICAgICAgICAKICAgICAgICB0cnk6CiAgICAgICAgICAgICMgMS4gTG9hZCB0aGUgUERCIGZpbGUgc3RydWN0dXJlICh0b3BvbG9neSBhbmQgcG9zaXRpb25zKQogICAgICAgICAgICBwZGIgPSBhcHAuUERCRmlsZShpbnB1dF9wYXRoKQogICAgICAgICAgICAKICAgICAgICAgICAgIyAyLiBQcmVwYXJlIHRoZSBUb3BvbG9neSAoQWRkIEh5ZHJvZ2VucyBpZiByZXF1ZXN0ZWQpCiAgICAgICAgICAgIGlmIGFkZF9oeWRyb2dlbnM6CiAgICAgICAgICAgICAgICBsb2dnZXIuaW5mbygiQWRkaW5nIG1pc3NpbmcgaHlkcm9nZW5zIChwcm90b25hdGlvbiBzdGF0ZSBAIHBIIDcuMCkuLi4iKQogICAgICAgICAgICAgICAgbW9kZWxsZXIgPSBhcHAuTW9kZWxsZXIocGRiLnRvcG9sb2d5LCBwZGIucG9zaXRpb25zKQogICAgICAgICAgICAgICAgbW9kZWxsZXIuYWRkSHlkcm9nZW5zKHNlbGYuZm9yY2VmaWVsZCwgcEg9Ny4wKQogICAgICAgICAgICAgICAgdG9wb2xvZ3kgPSBtb2RlbGxlci50b3BvbG9neQogICAgICAgICAgICAgICAgcG9zaXRpb25zID0gbW9kZWxsZXIucG9zaXRpb25zCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICB0b3BvbG9neSA9IHBkYi50b3BvbG9neQogICAgICAgICAgICAgICAgcG9zaXRpb25zID0gcGRiLnBvc2l0aW9ucwoKICAgICAgICAgICAgIyAzLiBDcmVhdGUgdGhlICdTeXN0ZW0nCiAgICAgICAgICAgICMgVGhlIFN5c3RlbSBvYmplY3QgY29ubmVjdHMgdGhlIFRvcG9sb2d5IChhdG9tcy9ib25kcykgdG8gdGhlIEZvcmNlZmllbGQgKHBoeXNpY3MgcnVsZXMpLgogICAgICAgICAgICAjIEl0IGNhbGN1bGF0ZXMgYWxsIGZvcmNlcyBhY3Rpbmcgb24gZXZlcnkgYXRvbS4KICAgICAgICAgICAgbG9nZ2VyLmRlYnVnKCJDcmVhdGluZyBPcGVuTU0gU3lzdGVtIChhcHBseWluZyBmb3JjZWZpZWxkIHBhcmFtZXRlcnMpLi4uIikKICAgICAgICAgICAgdHJ5OgogICAgICAgICAgICAgICAgc3lzdGVtID0gc2VsZi5mb3JjZWZpZWxkLmNyZWF0ZVN5c3RlbSgKICAgICAgICAgICAgICAgICAgICB0b3BvbG9neSwKICAgICAgICAgICAgICAgICAgICBub25ib25kZWRNZXRob2Q9YXBwLk5vQ3V0b2ZmLCAjIE5vIGN1dG9mZiBmb3IgdmFjdXVtL2ltcGxpY2l0IChjYWxjdWxhdGVzIEFMTCBwYWlyd2lzZSBpbnRlcmFjdGlvbnMpCiAgICAgICAgICAgICAgICAgICAgY29uc3RyYWludHM9YXBwLkhCb25kcywgICAgICAgIyBDb25zdHJhaW4gSHlkcm9nZW4gYm9uZCBsZW5ndGhzIChhbGxvd3MgbGFyZ2VyIHRpbWUgc3RlcHMgaW4gTUQpCiAgICAgICAgICAgICAgICAgICAgaW1wbGljaXRTb2x2ZW50PXNlbGYuc29sdmVudF9tb2RlbCAjIENvbnRpbnV1bSB3YXRlciBtb2RlbAogICAgICAgICAgICAgICAgKQogICAgICAgICAgICBleGNlcHQgVmFsdWVFcnJvciBhcyB2ZToKICAgICAgICAgICAgICAgICMgRmFsbGJhY2sgbG9naWMgZm9yIHdoZW4gaW1wbGljaXQgc29sdmVudCBmYWlscyAoY29tbW9uIHdpdGggc29tZSBmb3JjZWZpZWxkIGNvbWJvcykKICAgICAgICAgICAgICAgIGlmICJpbXBsaWNpdFNvbHZlbnQiIGluIHN0cih2ZSk6CiAgICAgICAgICAgICAgICAgICAgbG9nZ2VyLndhcm5pbmcoZiJJbXBsaWNpdCBTb2x2ZW50IHBhcmFtZXRlcnMgbm90IGZvdW5kIGZvciB0aGlzIGZvcmNlZmllbGQgY29uZmlndXJhdGlvbi4gVXNpbmcgVmFjdXVtIGVsZWN0cm9zdGF0aWNzIGluc3RlYWQgKHN0YW5kYXJkIGZhbGxiYWNrKS4iKQogICAgICAgICAgICAgICAgICAgIHN5c3RlbSA9IHNlbGYuZm9yY2VmaWVsZC5jcmVhdGVTeXN0ZW0oCiAgICAgICAgICAgICAgICAgICAgICAgIHRvcG9sb2d5LAogICAgICAgICAgICAgICAgICAgICAgICBub25ib25kZWRNZXRob2Q9YXBwLk5vQ3V0b2ZmLAogICAgICAgICAgICAgICAgICAgICAgICBjb25zdHJhaW50cz1hcHAuSEJvbmRzCiAgICAgICAgICAgICAgICAgICAgICAgICMgaW1wbGljaXRTb2x2ZW50IGRlZmF1bHRzIHRvIE5vbmUgKFZhY3V1bSkKICAgICAgICAgICAgICAgICAgICApCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIHJhaXNlIHZlCiAgICAgICAgICAgIAogICAgICAgICAgICAjIDQuIENyZWF0ZSB0aGUgSW50ZWdyYXRvcgogICAgICAgICAgICAjIEFuIEludGVncmF0b3IgaXMgdGhlIG1hdGggZW5naW5lIHRoYXQgbW92ZXMgYXRvbXMgYmFzZWQgb24gZm9yY2VzIChGPW1hKS4KICAgICAgICAgICAgIyAnTGFuZ2V2aW5JbnRlZ3JhdG9yJyBzaW11bGF0ZXMgYSBoZWF0IGJhdGggKGZyaWN0aW9uICsgcmFuZG9tIGNvbGxpc2lvbnMpIHRvIG1haW50YWluIHRlbXBlcmF0dXJlLgogICAgICAgICAgICAjCiAgICAgICAgICAgICMgRWR1Y2F0aW9uYWwgTm90ZToKICAgICAgICAgICAgIyBGb3IgcHVyZSBlbmVyZ3kgbWluaW1pemF0aW9uIChmaW5kaW5nIHRoZSBuZWFyZXN0IHZhbGxleSksIHdlIHRlY2huaWNhbGx5IGRvbid0IG5lZWQgYQogICAgICAgICAgICAjIHRoZXJtb3N0YXQgbGlrZSBMYW5nZXZpbiBiZWNhdXNlIHdlIGFyZW4ndCBzaW11bGF0aW5nIHRpbWUtcmVzb2x2ZWQgbW90aW9uIHlldC4KICAgICAgICAgICAgIyBIb3dldmVyLCBPcGVuTU0gcmVxdWlyZXMgYW4gSW50ZWdyYXRvciB0byBkZWZpbmUgdGhlIFNpbXVsYXRpb24gY29udGV4dC4KICAgICAgICAgICAgIyBJZiB3ZSB3ZXJlIHRvIHJ1biAic2ltdWxhdGlvbi5zdGVwKDEwMDApIiwgdGhpcyBpbnRlZ3JhdG9yIHdvdWxkIGdlbmVyYXRlCiAgICAgICAgICAgICMgcmVhbGlzdGljIEJyb3duaWFuIG1vdGlvbiwgc2ltdWxhdGluZyB0aGVybWFsIGZsdWN0dWF0aW9ucy4KICAgICAgICAgICAgaW50ZWdyYXRvciA9IG1tLkxhbmdldmluSW50ZWdyYXRvcigKICAgICAgICAgICAgICAgIDMwMCAqIHVuaXQua2VsdmluLCAgICAgICAjIFRlbXBlcmF0dXJlIChSb29tIHRlbXApCiAgICAgICAgICAgICAgICAxLjAgLyB1bml0LnBpY29zZWNvbmQsICAgIyBGcmljdGlvbiBjb2VmZmljaWVudAogICAgICAgICAgICAgICAgMi4wICogdW5pdC5mZW10b3NlY29uZHMgICMgVGltZSBzdGVwCiAgICAgICAgICAgICkKICAgICAgICAgICAgCiAgICAgICAgICAgICMgNS4gQ3JlYXRlIHRoZSBTaW11bGF0aW9uIGNvbnRleHQKICAgICAgICAgICAgc2ltdWxhdGlvbiA9IGFwcC5TaW11bGF0aW9uKHRvcG9sb2d5LCBzeXN0ZW0sIGludGVncmF0b3IpCiAgICAgICAgICAgIHNpbXVsYXRpb24uY29udGV4dC5zZXRQb3NpdGlvbnMocG9zaXRpb25zKQogICAgICAgICAgICAKICAgICAgICAgICAgIyBSZXBvcnQgRW5lcmd5IEJFRk9SRSBNaW5pbWl6YXRpb24KICAgICAgICAgICAgc3RhdGVfaW5pdGlhbCA9IHNpbXVsYXRpb24uY29udGV4dC5nZXRTdGF0ZShnZXRFbmVyZ3k9VHJ1ZSkKICAgICAgICAgICAgZV9pbml0ID0gc3RhdGVfaW5pdGlhbC5nZXRQb3RlbnRpYWxFbmVyZ3koKS52YWx1ZV9pbl91bml0KHVuaXQua2lsb2pvdWxlc19wZXJfbW9sZSkKICAgICAgICAgICAgbG9nZ2VyLmluZm8oZiJJbml0aWFsIFBvdGVudGlhbCBFbmVyZ3k6IHtlX2luaXQ6LjJmfSBrSi9tb2wiKQogICAgICAgICAgICAKICAgICAgICAgICAgaWYgZV9pbml0ID4gMWU2OgogICAgICAgICAgICAgICAgbG9nZ2VyLmluZm8oIiAgLT4gSGlnaCBpbml0aWFsIGVuZXJneSBkZXRlY3RlZCBkdWUgdG8gc3RlcmljIGNsYXNoZXMuIE1pbmltaXphdGlvbiB3aWxsIHJlc29sdmUgdGhpcy4iKQogICAgICAgICAgICAKICAgICAgICAgICAgIyA2LiBSdW4gRW5lcmd5IE1pbmltaXphdGlvbiAoR3JhZGllbnQgRGVzY2VudCkKICAgICAgICAgICAgbG9nZ2VyLmluZm8oIk1pbmltaXppbmcgZW5lcmd5Li4uIikKICAgICAgICAgICAgc2ltdWxhdGlvbi5taW5pbWl6ZUVuZXJneSgpCiAgICAgICAgICAgIAogICAgICAgICAgICAjIFJlcG9ydCBFbmVyZ3kgQUZURVIgTWluaW1pemF0aW9uCiAgICAgICAgICAgIHN0YXRlX2ZpbmFsID0gc2ltdWxhdGlvbi5jb250ZXh0LmdldFN0YXRlKGdldEVuZXJneT1UcnVlLCBnZXRQb3NpdGlvbnM9VHJ1ZSkKICAgICAgICAgICAgZV9maW5hbCA9IHN0YXRlX2ZpbmFsLmdldFBvdGVudGlhbEVuZXJneSgpLnZhbHVlX2luX3VuaXQodW5pdC5raWxvam91bGVzX3Blcl9tb2xlKQogICAgICAgICAgICBsb2dnZXIuaW5mbyhmIkZpbmFsIFBvdGVudGlhbCBFbmVyZ3k6ICAge2VfZmluYWw6LjJmfSBrSi9tb2wiKQogICAgICAgICAgICAKICAgICAgICAgICAgIyA3LiBTYXZlIFJlc3VsdAogICAgICAgICAgICB3aXRoIG9wZW4ob3V0cHV0X3BhdGgsICd3JykgYXMgZjoKICAgICAgICAgICAgICAgIGFwcC5QREJGaWxlLndyaXRlRmlsZShzaW11bGF0aW9uLnRvcG9sb2d5LCBzdGF0ZV9maW5hbC5nZXRQb3NpdGlvbnMoKSwgZikKICAgICAgICAgICAgICAgIAogICAgICAgICAgICByZXR1cm4gVHJ1ZQoKICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgIGxvZ2dlci5lcnJvcihmIlBoeXNpY3Mgc2ltdWxhdGlvbiBmYWlsZWQ6IHtlfSIpCiAgICAgICAgICAgIGlmICJ0ZW1wbGF0ZSIgaW4gc3RyKGUpLmxvd2VyKCk6CiAgICAgICAgICAgICAgICBsb2dnZXIuZXJyb3IoIkVycm9yIEhpbnQ6IE9wZW5NTSBjb3VsZG4ndCBtYXRjaCByZXNpZHVlcyB0byB0aGUgZm9yY2VmaWVsZC4gVGhpcyB1c3VhbGx5IG1lYW5zIGF0b21zIGFyZSBtaXNzaW5nIG9yIG5hbWVkIGluY29ycmVjdGx5LiIpCiAgICAgICAgICAgIHJldHVybiBGYWxzZQo="
patch_content = base64.b64decode(encoded_patch).decode("utf-8")

with open(physics_file, "w") as f:
    f.write(patch_content)

print("✅ Hotfix applied: OpenMM import is now optional.")

# Force reload
import importlib
importlib.reload(synth_pdb)
try:
    import synth_pdb.physics
    importlib.reload(synth_pdb.physics)
except ImportError:
    pass


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
from synth_pdb.generator import generate_pdb_content
from synth_pdb.relaxation import calculate_relaxation_rates, predict_order_parameters
import biotite.structure.io.pdb as pdb
import io

# 1. Generate a Test Protein (Zinc Finger-like Motif)
# Visualization: Beta Hairpin (Sheet-Turn-Sheet) + Alpha Helix
# This provides a rich mix of rigid and flexible regions.
sequence = "VKITVGGTLTVALGGALALALALALALAA"
structure_def = "1-5:beta,6-8:random,9-13:beta,14-16:random,17-29:alpha"

print("🧬 Generating virtual protein (Zinc Finger-like fold)...")
print("   - Optimizing side-chains (Monte Carlo)...")
print("   - Minimizing energy (OpenMM if available)...")

pdb_content = generate_pdb_content(
    sequence_str=sequence, 
    structure=structure_def, 
    optimize_sidechains=True, 
    minimize_energy=True
)
pdb_file = pdb.PDBFile.read(io.StringIO(pdb_content))
structure = pdb_file.get_structure(model=1)

# Pre-calculate Order Parameters (S2) based on structure
s2_map = predict_order_parameters(structure)
res_ids = sorted(list(s2_map.keys()))
s2_values = [s2_map[r] for r in res_ids]

print("✅ Protein Model Ready!")


### 🧬 The Simulated Protein (Zinc Finger Motif)

We have generated a synthetic **Zinc Finger-like** fold to demonstrate contrast between different secondary structures:
*   **Residues 1-5**: Beta Strand 1 (Rigid, Extended)
*   **Residues 6-8**: Turn (Flexible)
*   **Residues 9-13**: Beta Strand 2 (Rigid, Extended)
*   **Residues 14-16**: Loop (Flexible)
*   **Residues 17-29**: Alpha Helix (Rigid, Compact)

Notice how the physics engine (`synth-pdb`) automatically assigns different **Order Parameters ($S^2$)** to these regions. $S^2$ represents spatial restriction: **1.0** is completely rigid, **0.0** is completely disordered.

In [ ]:
# Visualizing the Structure
import py3Dmol

view = py3Dmol.view(width=400, height=300)
view.addModel(pdb_content, 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
print("👀 3D View of Helix-Turn-Helix (Blue=N-term, Red=C-term)")
view.show()

### 📊 Guide to the Plots

When you run the simulator below, you will see three coupled plots. Here is how to interpret them:

1.  **$R_1$ (Longitudinal Rate - Blue)**: 
    *   *Physics*: Sensitive to fast interaction fluctuations (nanosecond scale).
    *   *Pattern*: Often relatively flat for folded proteins, but may dip in flexible loops.

2.  **$R_2$ (Transverse Rate - Red)**: 
    *   *Physics*: Sensitive to slow motions (global tumbling) and chemical exchange.
    *   *Key Insight*: **$R_2$ scales with protein size**. Large proteins (slow tumbling) have high $R_2$ rates. High $R_2$ means the signal decays fast (broad lines), making large proteins hard to study!
    *   *Flexible Regions*: $R_2$ drops sharply because local flexibility averages out the magnetic interactions.

3.  **Heteronuclear NOE (Green)**: 
    *   *The Rigidity Sensor*: This is the most robust indicator of local structure.
    *   **Value ~ 0.8**: Rigid backbone (Helix/Sheet).
    *   **Value < 0.6**: Flexible loop/terminus.
    *   **Negative Values**: Extremely flexible or unfolded.

---

In [ ]:
def plot_relaxation(field_mhz=600, tau_m_ns=10.0):
    """Calculate and plot rates for given conditions."""
    
    # Calculate Rates using synth-pdb physics engine
    rates = calculate_relaxation_rates(
        structure, 
        field_mhz=field_mhz, 
        tau_m_ns=tau_m_ns, 
        s2_map=s2_map
    )
    
    r1 = [rates[r]['R1'] for r in res_ids]
    r2 = [rates[r]['R2'] for r in res_ids]
    noe = [rates[r]['NOE'] for r in res_ids]
    
    # Plotting
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    
    # R1 Plot
    ax1.plot(res_ids, r1, 'o-', color='blue', label='R1 (Longitude)')
    ax1.set_ylabel('$R_1$ ($s^{-1}$)')
    ax1.set_title(f'Relaxation Dynamics @ {field_mhz} MHz, $\\tau_m$={tau_m_ns}ns')
    ax1.grid(True, alpha=0.3)
    
    # R2 Plot
    ax2.plot(res_ids, r2, 's-', color='red', label='R2 (Transverse)')
    ax2.set_ylabel('$R_2$ ($s^{-1}$)')
    ax2.grid(True, alpha=0.3)
    
    # NOE Plot
    ax3.plot(res_ids, noe, '^-', color='green', label='HetNOE')
    ax3.set_ylabel('NOE Ratio')
    ax3.set_xlabel('Residue Number')
    ax3.grid(True, alpha=0.3)
    
    # Highlight the flexible loop region (7-9)
    for ax in [ax1, ax2, ax3]:
        ax.axvspan(7, 9, color='yellow', alpha=0.2, label='Flexible Loop')
        ax.legend()
        
    plt.show()

# Create Interactive Widgets
print("🎛 Starting Virtual Spectrometer...")
interact(
    plot_relaxation, 
    field_mhz=widgets.IntSlider(min=400, max=1200, step=100, value=600, description='Field (MHz)'),
    tau_m_ns=widgets.FloatSlider(min=2.0, max=50.0, step=1.0, value=10.0, description='Tumbling (ns)')
);

### 🧪 Try These Experiments

Use the sliders above to change the "experimental" conditions:

1.  **Simulate a Giant Protein Complex**:
    *   Slide **Tumbling (ns)** to **40.0 ns**.
    *   *Observation*: Look at the **$R_2$ (Red)** plot. It shoots up significantly! This is why NMR is limited to smaller proteins (< 50-100 kDa) without special tricks (like TROSY).
    *   *Note*: The NOE profile stays mostly the same—local rigidity hasn't changed, only the global tumbling.

2.  **Go to High Field**:
    *   Slide **Field (MHz)** from **600** to **1200**.
    *   *Observation*: $R_2$ increases. This is due to **Chemical Shift Anisotropy (CSA)** becoming a stronger relaxation mechanism at high magnetic fields.
